# overall_rating

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

import folders

In [ ]:
def get_y(path):
    with open(path, 'r', encoding='utf-8') as fd:
        # Read column names.
        header = fd.readline()[:-1]
        column_names = header.split('\t')
        for i, column_name in enumerate(column_names):
            if i % 2 == 1 and column_name[:-5] != column_names[i - 1][:-5]:
                raise ValueError('Mismatched column names: {} and {}'.format(column_name, column_names[i - 1]))
                
        # Read values.
        n_categories = len(column_names)//2
        y_true, y_pred = [[] for _ in range(n_categories)], [[] for _ in range(n_categories)]
        line = fd.readline()[:-1]
        while len(line) > 0:
            values = [int(part) for part in line.split('\t')]
            for i, value in enumerate(values):
                j = i//2
                if i % 2 == 0:
                    y_true[j].append(value)
                else:
                    y_pred[j].append(value)
            line = fd.readline()[:-1]
    return np.array(y_true), np.array(y_pred)

In [ ]:
def get_overall(y):
    return [max(y[:7, i]) for i in range(y.shape[1])]

In [ ]:
def print_model_accuracies(model_path, latex=False):
    fnames = os.listdir(model_path)
    for fname in fnames:
        path = os.path.join(model_path, fname)
        y_true, y_pred = get_y(path)
        if y_true.shape[1] == 0:
            continue
        print('{}'.format(path), end='')
        category_accuracies = []
        for i in range(len(y_true)):
            accuracy = accuracy_score(y_true[i], y_pred[i])
            category_accuracies.append(accuracy)
        avg_accuracy = sum(category_accuracies)/len(category_accuracies)
        overall_true = get_overall(y_true)
        overall_pred = get_overall(y_pred)
        overall_accuracy = accuracy_score(overall_true, overall_pred)
        if latex:
            for accuracy in category_accuracies:
                print(' & {:.4f}'.format(accuracy), end='')
            print(' & {:.4f}'.format(avg_accuracy), end='')
            print(' & {:.4f}'.format(overall_accuracy), end='')
            print('\\\\')
        else:
            for accuracy in category_accuracies:
                print(' {:.4f}'.format(accuracy), end='')
            print(' {:.4f}'.format(avg_accuracy), end='')
            print(' {:.4f}'.format(overall_accuracy))

In [ ]:
model_names = os.listdir(folders.PREDICTIONS_PATH)
for model_name in model_names:
    if model_name == 'baselines':
        baselines_path = os.path.join(folders.PREDICTIONS_PATH, model_name)
        baseline_model_names = os.listdir(baselines_path)
        for baseline_model_name in baseline_model_names:
            baseline_model_path = os.path.join(baselines_path, baseline_model_name)
            print_model_accuracies(baseline_model_path, latex=True)
        continue
    model_path = os.path.join(folders.PREDICTIONS_PATH, model_name)
    print_model_accuracies(model_path, latex=True)